# PYTHON YANNICK BOUTY - ZOOV - DATA ENGINEER INTERNSHIP

## Chargement des données

In [126]:
import folium
import json
import pandas as pd

#Chargement des données
coordonnees_departements_france = pd.read_json('https://france-geojson.gregoiredavid.fr/repo/departements.geojson')
coordonnees_departements_france_geajson = 'https://france-geojson.gregoiredavid.fr/repo/departements.geojson'
donnees_covid_departements_france = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/63352e38-d353-4b54-bfd1-f1b3ee1cabd7',sep=';')


## Découverte des données

In [84]:
#Visualisation des données coordonnées
coordonnees_departements_france.head()

,type,features
0,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
1,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
2,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
3,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
4,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly..."


In [85]:
#Visualisation des données covid
donnees_covid_departements_france.head()

,dep,sexe,jour,hosp,rea,rad,dc
0,01,0,2020-03-18,2,0,1,0
1,01,1,2020-03-18,1,0,1,0
2,01,2,2020-03-18,1,0,0,0
3,02,0,2020-03-18,41,10,18,11
4,02,1,2020-03-18,19,4,11,6


In [86]:
#Visualisation des clés coordonnées
coordonnees_departements_france.keys()

Index(['type', 'features'], dtype='object')

In [87]:
#Visualisation des clés coordonnées/features
coordonnees_departements_france["features"][0].keys()

dict_keys(['type', 'geometry', 'properties'])

In [146]:
#Visualisation des clés coordonnées/features/properties
coordonnees_departements_france["features"][0]["properties"]

{'code': '02', 'nom': 'Aisne'}

In [89]:
#Visualisation des clés Covid
donnees_covid_departements_france.keys()

Index(['dep', 'sexe', 'jour', 'hosp', 'rea', 'rad', 'dc'], dtype='object')

In [90]:
#Nombre de lignes et de colonnes coordonnées
coordonnees_departements_france.shape

(96, 2)

In [91]:
#Nombre de lignes et de colonnes covid
donnees_covid_departements_france.shape

(86658, 7)

In [92]:
#Détails des données coordonnées
coordonnees_departements_france.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   type      96 non-null     object
 1   features  96 non-null     object
dtypes: object(2)
memory usage: 1.6+ KB


In [93]:
#Détails des données covid
donnees_covid_departements_france.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86658 entries, 0 to 86657
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dep     86658 non-null  object
 1   sexe    86658 non-null  int64 
 2   jour    86658 non-null  object
 3   hosp    86658 non-null  int64 
 4   rea     86658 non-null  int64 
 5   rad     86658 non-null  int64 
 6   dc      86658 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 4.6+ MB


In [94]:
#Description des données coordonnées.
coordonnees_departements_france.describe()

,type,features
count,96,96
unique,1,96
top,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Mult..."
freq,96,1


In [95]:
#Description des données covid
donnees_covid_departements_france.describe()

,sexe,hosp,rea,rad,dc
count,86658.000000,86658.000000,86658.000000,86658.000000,86658.000000
mean,1.000000,103.291410,14.611796,583.153696,139.109049
std,0.816501,195.210618,35.560856,994.147168,238.576167
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,9.000000,1.000000,83.000000,17.000000
50%,1.000000,34.000000,3.000000,229.000000,50.000000
75%,2.000000,109.000000,13.000000,598.000000,142.000000
max,2.000000,3281.000000,855.000000,11083.000000,2649.000000


In [96]:
#Y-a-t-il des valeurs null dans les coordonnées ?
coordonnees_departements_france.isna().sum()

type        0
features    0
dtype: int64

In [97]:
#Y-a-t-il des valeurs null dans les données covid ?
donnees_covid_departements_france.isna().sum()

dep     0
sexe    0
jour    0
hosp    0
rea     0
rad     0
dc      0
dtype: int64

In [98]:
#Corrélation entre les données covid ?
donnees_covid_departements_france.corr()

,sexe,hosp,rea,rad,dc
sexe,1.000000,-0.172203,-0.185025,-0.187875,-0.211642
hosp,-0.172203,1.000000,0.897850,0.638368,0.670321
rea,-0.185025,0.897850,1.000000,0.494910,0.523017
rad,-0.187875,0.638368,0.494910,1.000000,0.964946
dc,-0.211642,0.670321,0.523017,0.964946,1.000000


## Visualisation des hospitalisations par département

In [168]:
#Préparation des données covid
#Supression des colonnes non utiles à cette visualisation
droped_colonnes = ['sexe','jour','rea','rad','dc']
hospitalisation_covid = donnees_covid_departements_france.drop(columns=droped_colonnes)
#Sommes des hospitalisations par département
hospitalisation_covid = hospitalisation_covid.groupby('dep', as_index=False).sum()
#Suppression des départements non présents dans le GeoJson
index971 = hospitalisation_covid[hospitalisation_covid['dep'] == '971' ].index
hospitalisation_covid.drop(index971, inplace=True)
index972 = hospitalisation_covid[hospitalisation_covid['dep'] == '972' ].index
hospitalisation_covid.drop(index972, inplace=True)
index973 = hospitalisation_covid[hospitalisation_covid['dep'] == '973' ].index
hospitalisation_covid.drop(index973, inplace=True)
index974 = hospitalisation_covid[hospitalisation_covid['dep'] == '974' ].index
hospitalisation_covid.drop(index974, inplace=True)
index976 = hospitalisation_covid[hospitalisation_covid['dep'] == '976' ].index
hospitalisation_covid.drop(index976, inplace=True)

In [170]:
#Affichage de la carte
france = folium.Map(location=[46.539758, 2.430331], zoom_start=6)
folium.Choropleth(
    geo_data = coordonnees_departements_france_geajson,
    data = hospitalisation_covid,
    name = 'Hospitalisation',
    columns = ['dep', 'hosp'],
    key_on = 'feature.properties.code',
    fill_color='OrRd',
    legend_name='Hospitalisations par departement'
).add_to(france)

folium.LayerControl().add_to(france)

france

## Visualisation du nombre de personnes en réanimation par département

In [166]:
#Préparation des données covid
#Supression des colonnes non utiles à cette visualisation
droped_colonnes = ['sexe','jour','hosp','rad','dc']
reanimation_covid = donnees_covid_departements_france.drop(columns=droped_colonnes)
#Sommes des hospitalisations par département
reanimation_covid = reanimation_covid.groupby('dep', as_index=False).sum()
#Suppression des départements non présents dans le GeoJson
index971 = reanimation_covid[reanimation_covid['dep'] == '971' ].index
reanimation_covid.drop(index971, inplace=True)
index972 = reanimation_covid[reanimation_covid['dep'] == '972' ].index
reanimation_covid.drop(index972, inplace=True)
index973 = reanimation_covid[reanimation_covid['dep'] == '973' ].index
reanimation_covid.drop(index973, inplace=True)
index974 = reanimation_covid[reanimation_covid['dep'] == '974' ].index
reanimation_covid.drop(index974, inplace=True)
index976 = reanimation_covid[reanimation_covid['dep'] == '976' ].index
reanimation_covid.drop(index976, inplace=True)

In [167]:
#Affichage de la carte
france = folium.Map(location=[46.539758, 2.430331], zoom_start=6)
folium.Choropleth(
    geo_data = coordonnees_departements_france_geajson,
    data = reanimation_covid,
    name = 'Réanimation',
    columns = ['dep', 'rea'],
    key_on = 'feature.properties.code',
    fill_color='OrRd',
    legend_name='Nombre de personnes en reanimation par departement'
).add_to(france)

folium.LayerControl().add_to(france)

france